In [1]:
!pip install orkg
!pip install openml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 8.0 MB/s eta 0:00:00
  Created wheel for cardinality: filename=cardinality-0.1.1-py3-none-any.whl size=2587 sha256=af49b3c71f6c704d0ba76f57ddf17a4514cb31e39b5db5ab0b1eae27e6cd0a5b
  Stored in directory: /root/.cache/pip/wheels/b8/19/d1/2665c004b583a7d1880fa59055a3e462d6e35841a01b57010b
  Created wheel for hammock: filename=hammock-0.2.4-py3-none-any.whl size=2406 sha256=d6f337723a4d7c4024de9b18c0a614f3a16c6e2376e8312b0887b1d87cfb48dc
  Stored in directory: /root/.cache/pip/wheels/f3/55/5b/e1e1e4366a23623af753c68c926db14ea6e58c0f038

# **Define variables**

In [2]:
custom_classID = 'C72277' # You need to edit it.
dataset_name_label = 'Class Test3'  # You need to edit it.
dataset_name='ilpd'
hasFeatures_predicate_id = 'P114013'
feature_predicates = {
    'Number of instances': 'P39021',
    'Number of features': 'P72007',
    'Number of classes': 'P52036',
    'Standard Deviation Ratio': 'P114000',
    'Class Entropy': 'P114002',
    'Normal Entropy': 'P114003',
    'hasFeature': 'P114013'
}

feature_predicates_2 = {
    'Skewness': 'P59063',
    'Kurtosis': 'P59064',
    'Min values': 'P44107',
    'Max values': 'P44108',
    'Mean values': 'P67072',
    'Median values': 'P57006',
}
possible_targets = ['Class', 'class', 'Defects', 'c', 'Author', 'band_type', 'defects', 'Prevention', 'problems', 'binaryClass', 'y', 'character', 'attribute_21', 'target', 'Result', 'Phase', 'result']

automl_datasets = ['kr-vs-kp', 'letter', 'balance-scale', 'mfeat-factors', 'mfeat-fourier', 'breast-w',
                   'mfeat-karhunen', 'mfeat-morphological', 'mfeat-zernike', 'optdigits', 'credit-approval',
                   'credit-g', 'pendigits', 'diabetes', 'sick', 'spambase', 'splice', 'tic-tac-toe', 'vehicle',
                   'electricity', 'satimage',  'isolet', 'vowel', 'analcatdata_authorship',
                   'analcatdata_dmft', 'mnist_784', 'pc4', 'pc3', 'jm1', 'kc2', 'kc1', 'pc1', 'bank-marketing',
                   'banknote-authentication', 'blood-transfusion-service-center', 'cnae-9', 'first-order-theorem-proving',
                   'har', 'ilpd', 'madelon', 'nomao', 'ozone-level-8hr', 'phoneme', 'qsar-biodeg', 'wall-robot-navigation',
                   'semeion', 'wdbc', 'adult', 'Bioresponse', 'PhishingWebsites', 'GesturePhaseSegmentationProcessed',
                   'cylinder-bands', 'dresses-sales', 'numerai28.6', 'texture', 'dna', 'churn',
                   'Devnagari-Script', 'CIFAR_10', 'MiceProtein', 'car', 'Internet-Advertisements', 'mfeat-pixel',
                   'steel-plates-fault', 'wilt', 'segment', 'climate-model-simulation-crashes', 'Fashion-MNIST',
                   'jungle_chess_2pcs_raw_endgame_complete', 'JapaneseVowels']



In [3]:
from google.colab import userdata
email = userdata.get('email')
password = userdata.get('password')

In [4]:
from orkg import ORKG, Hosts

orkg = ORKG(host=Hosts.SANDBOX, creds=(email, password))

# **Create the Class with the CustomID**

In [5]:
classes = orkg.classes.add(id=custom_classID, label=f'{dataset_name_label} Test').content
classes

{'id': 'C72277',
 'label': 'Class Test3 Test',
 'uri': None,
 'description': None,
 'created_at': '2024-03-19T20:59:45.215190356+01:00',
 'created_by': 'ba7b42b0-dbeb-41fb-99d5-cb6b4140660e',
 'modifiable': True,
 '_class': 'class'}

# **Extract the Meta-Features from the Datasets**

# **Adding meta features that having single value to ORKG resources**

In [6]:
import pandas as pd
import numpy as np
from sklearn import datasets
from scipy.stats import entropy
import time
def extract_meta_features(dataset_name):
    # Fetch the dataset from OpenML
    dataset = datasets.fetch_openml(dataset_name, parser='auto', as_frame=True)

    # Identify the target variable
    possible_targets = ['Class', 'class', 'Defects', 'c', 'Author', 'band_type', 'defects', 'Prevention', 'problems', 'binaryClass', 'y', 'character', 'attribute_21', 'target', 'Result', 'Phase', 'result']
    target = next((t for t in possible_targets if t in dataset.frame.columns), None)
    if target is None:
        raise ValueError("Target variable not found in the dataset.")

    # Extract meta-features
    num_instances = dataset.frame.shape[0]
    num_features = dataset.frame.shape[1]
    num_classes = len(dataset.frame[target].unique())

    num_instances = int(num_instances * 1.1) # Adding 10% to num_instances for testen
    num_features = int(num_features * 1.1)
    num_classes = int (num_classes * 1.1)
    meta_features = {
        'Number of instances': num_instances,
        'Number of features': num_features,
        'Number of classes': num_classes,
        'Standard Deviation Ratio': None,
        'Class Entropy': None,
        'Normal Entropy': None,
        'hasFeatures': {}
    }
    time.sleep(1)
    for feature in dataset.frame.select_dtypes(include=np.number).columns:
        meta_features['hasFeatures'][feature] = {
            'Skewness': round(dataset.frame[feature].skew(), 2),
            'Kurtosis': round(dataset.frame[feature].kurt(), 2),
            'Min values': round(dataset.frame[feature].min(), 2),
            'Max values': round(dataset.frame[feature].max(), 2),
            'Mean values': round(dataset.frame[feature].mean(), 2),
            'Median values': round(dataset.frame[feature].median(), 2)
        }

    # Calculate other meta-features
    sd_ratio = round((dataset.frame.std(numeric_only=True) / dataset.frame.mean(numeric_only=True)).mean(), 2)
    class_entropy = round(entropy(dataset.frame[target].value_counts(normalize=True), base=2), 2)
    normal_entropy = round(class_entropy / np.log2(len(dataset.frame[target].unique())), 2)
    meta_features['Standard Deviation Ratio'] = sd_ratio
    meta_features['Class Entropy'] = class_entropy.item()
    meta_features['Normal Entropy'] = normal_entropy

    return meta_features


# **Process the Meta Features**

In [7]:
def process_meta_features(data):
    features_resource_ids = []

    # Iterate through the features in data['Features']
    for feature_name, feature_data in data['hasFeatures'].items():
        # Add feature as resource
        feature_resource = orkg.resources.add(label=feature_name).content
        feature_resource_id = feature_resource['id']
        features_resource_ids.append(feature_resource_id)

        # Add feature data as properties
        for meta_feature, value in feature_data.items():
            # add Literal
            literal_id = orkg.literals.add(label=str(value)).content['id']
            # add the predicate
            predicate_id = feature_predicates_2[meta_feature]

            # connect them
            statement_data = {
                'subject_id': feature_resource_id,
                'predicate_id': predicate_id,
                'object_id': literal_id
            }
            orkg.statements.add(**statement_data)

    return features_resource_ids

# **Add the Meta features to the ORKG**

In [ ]:
for dataset in automl_datasets:
    # Add dataset as a resource
    resource = orkg.resources.add(label=f'{dataset} test', classes=[custom_classID]).content
    resource_id = resource['id']

    # Extract meta features for the current dataset
    data = extract_meta_features(dataset)
    time.sleep(1)
    ids = process_meta_features(data)

    # Construct dictionary of single value meta-features
    single_value_metaFeatures = {
        'Number of instances': data['Number of instances'],
        'Number of features': data['Number of features'],
        'Number of classes': data['Number of classes'],
        'Standard Deviation Ratio': data['Standard Deviation Ratio'],
        'Class Entropy': data['Class Entropy'],
        'Normal Entropy': data['Normal Entropy'],
        'hasFeature': ids
    }

    print(f"Adding {dataset}")
    time.sleep(1)
    # Iterate through single_value_metaFeatures
    for feature, value in single_value_metaFeatures.items():
        # Check if the feature is 'hasFeature'
        if feature == 'hasFeature':
            # Connect each feature resource ID to the dataset resource
            for feature_resource_id in value:
                statement_data = {
                    'subject_id': resource_id,
                    'predicate_id': hasFeatures_predicate_id,
                    'object_id': feature_resource_id
                }
                orkg.statements.add(**statement_data)
        else:
            # Add Literal
            literal_id = orkg.literals.add(label=str(value)).content['id']
            # Add the predicate
            predicate_id = feature_predicates[feature]

            # Connect them
            statement_data = {
                'subject_id': resource_id,
                'predicate_id': predicate_id,
                'object_id': literal_id
            }
            orkg.statements.add(**statement_data)

    print(f"Done {dataset}")


Adding kr-vs-kp
Done kr-vs-kp


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name letter exist. Versions may be fundamentally different, returning version 1.
  warn(


Adding letter
Done letter


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name balance-scale exist. Versions may be fundamentally different, returning version 1.
  warn(


Adding balance-scale
Done balance-scale


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name mfeat-factors exist. Versions may be fundamentally different, returning version 1.
  warn(
